<a href="https://colab.research.google.com/github/Royal1shrestha/Google_Collab/blob/main/Reccomender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df_rating = pd.read_csv('/content/sample_data/Books/BX-Book-Ratings.csv', sep=";", error_bad_lines=False, encoding='latin1')
df_books = pd.read_csv('/content/sample_data/Books/BX-Books.csv', sep=";", error_bad_lines=False, encoding='latin1')
df_users = pd.read_csv('/content/sample_data/Books/BX-Users.csv', sep=";", error_bad_lines=False, encoding='latin1')

In [ ]:
df_rating.head(2)
# df_books.head(2)
# df_users.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [ ]:
df_images = df_books.iloc[:, -2:]
df_books = df_books.iloc[:, :-2]
df_books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [ ]:
userCounts = df_rating['User-ID'].value_counts()
print(userCounts)

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: User-ID, Length: 105283, dtype: int64


In [ ]:
df_books.isnull().sum()
# df_rating.isnull().sum()
# df_users.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
dtype: int64

In [ ]:
df_books['Book-Author'].fillna(df_books['Book-Author'].mode()[0], inplace=True)
df_books['Publisher'].fillna(df_books['Publisher'].mode()[0], inplace=True)

In [ ]:
dfr = df_rating
x = userCounts > 200
y = x[x].index
dfr = dfr[dfr['User-ID'].isin(y)]
dfr.head()

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [ ]:
ratings_with_books = dfr.merge(df_books, on="ISBN")
ratings_with_books.head(2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [ ]:
num_ratings = ratings_with_books.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().rename(columns={'Book-Rating': 'totalRatingCount'})
num_ratings.head(3)

,Book-Title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1


In [ ]:
final_ratings = ratings_with_books.merge(num_ratings,on="Book-Title")
final_ratings.head(3)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,totalRatingCount
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [ ]:
final_ratings = final_ratings[final_ratings['totalRatingCount']>=50]
final_ratings.head(3)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,totalRatingCount
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [ ]:
final_ratings.drop_duplicates(['User-ID','Book-Title'], inplace=True)

In [ ]:
final_ratings.shape

(59850, 9)

In [ ]:
book_pivot = final_ratings.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating')
book_pivot

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [ ]:
book_pivot.shape

(742, 888)

In [ ]:
book_pivot.fillna(0,inplace=True)
book_pivot

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Above matrix is a sparse matrix
# CSR matrix
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
random_index = np.random.choice(book_sparse.shape[0])
distance, suggestion = model_knn.kneighbors(book_pivot.iloc[random_index,:].values.reshape(1,-1), n_neighbors=5)

In [ ]:
distance

array([[0.        , 0.46475172, 0.51854218, 0.57174192, 0.61377115]])

In [ ]:
suggestion

array([[524, 185, 386, 734, 149]])

In [ ]:
book_pivot.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=742)

In [ ]:
book_name = book_pivot.index

In [ ]:
for i in range(len(suggestion)):
  print(book_pivot.index[suggestion[i]])

Index(['The Reader', 'Drowning Ruth', 'Jewel',
       'The Red Tent (Bestselling Backlist)',
       'A Lesson Before Dying (Vintage Contemporaries (Paperback))'],
      dtype='object', name='Book-Title')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
pickle.dump(model_knn, open('drive/MyDrive/Model/model_knn.pkl','wb'))
pickle.dump(book_name, open('drive/MyDrive/Model/book_name.pkl','wb'))
pickle.dump(final_ratings, open('drive/MyDrive/Model/final_ratings.pkl','wb'))
pickle.dump(book_pivot, open('drive/MyDrive/Model/book_pivot.pkl','wb'))

NameError: name 'model_knn' is not defined

In [ ]:
def get_recommend(book_name):
  book_id = np.where(book_pivot.index == book_name)[0][0]
  distance, suggestion = model_knn.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=5)

  for i in range(len(suggestion)):
    books = book_pivot.index[suggestion[i]]
    for j in books:
      print(j)

In [ ]:
get_recommend("The Queen of the Damned (Vampire Chronicles (Paperback))")

The Queen of the Damned (Vampire Chronicles (Paperback))
The Vampire Lestat (Vampire Chronicles, Book II)
The Tale of the Body Thief (Vampire Chronicles (Paperback))
The Witching Hour (Lives of the Mayfair Witches)
Interview with the Vampire


In [ ]:
def get_recommend_with_distance(book_name):
  book_id = np.where(book_pivot.index == book_name)[0][0]
  distance, suggestion = model_knn.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=5)

  for i in range(len(suggestion)):
    books = book_pivot.index[suggestion[i]]
    books_with_distance = zip(books, distance[i])
    sorted_books_with_distance = sorted(books_with_distance, key=lambda x: x[1])  # Sort by distance
    for book, dist in sorted_books_with_distance:
        print(f"{book}, Distance: {dist}")

In [ ]:
get_recommend_with_distance("The Queen of the Damned (Vampire Chronicles (Paperback))")

The Queen of the Damned (Vampire Chronicles (Paperback)), Distance: 2.220446049250313e-16
The Vampire Lestat (Vampire Chronicles, Book II), Distance: 0.5178411864186413
The Tale of the Body Thief (Vampire Chronicles (Paperback)), Distance: 0.5298544346433054
The Witching Hour (Lives of the Mayfair Witches), Distance: 0.7371460814604434
Interview with the Vampire, Distance: 0.738368248476658


In [ ]:
book_pivot = pickle.load(open('/content/drive/MyDrive/Model/book_pivot.pkl','rb'))
final_ratings = pickle.load(open('/content/drive/MyDrive/Model/final_ratings.pkl','rb'))
model_knn = pickle.load(open('/content/drive/MyDrive/Model/book_pivot.pkl','rb'))

In [ ]:
# Function to calculate precision, recall, and F1-score
def calculate_metrics(model, X, actual_ratings):
    true_positives = 0
    false_positives = 0
    total_relevant = 0

    for i in range(X.shape[0]):
        book_name = X.index[i]
        book_id = np.where(book_pivot.index == book_name)[0][0]
        distance, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=3)
        recommended_books = book_pivot.index[suggestion[0]]

        # Count true positives and false positives
        for recommended_book in recommended_books:
            if recommended_book in actual_ratings['Book-Title'].values:
                total_relevant += 1
                if recommended_book == book_name:
                    true_positives += 1
            else:
                false_positives += 1

    # Calculate precision, recall, and F1-score
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / total_relevant
    f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

actual_ratings_df = final_ratings[['Book-Title', 'Book-Rating']]

# Calculate precision, recall, and F1-score for the model
precision, recall, f1_score = calculate_metrics(model_knn, book_pivot, actual_ratings_df)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)


Precision: 1.0
Recall: 0.3333333333333333
F1-score: 0.5
